In [ ]:
import numpy as np

In [ ]:
import os
# Download the Armadillo from github
mesh_path = "armadillo.obj"
if not os.path.exists(mesh_path):
    import wget
    url = 'https://raw.githubusercontent.com/alecjacobson/common-3d-test-models/refs/heads/master/data/armadillo.obj'
    filename = wget.download(url)
    print()
    print(filename)
else:
    print("Found .obj file")
    
def sample_positions(n, random_sampling, sdf):
    if random_sampling:
        U = (np.random.rand(n*n*n, 3)-0.5)*2
    else:
        gx, gy, gz = np.meshgrid(np.linspace(-1.0, 1.0, n+1), np.linspace(-1.0, 1.0, n+1), np.linspace(-1.0, 1.0, n+1))
        U = np.vstack((gx.flatten(), gy.flatten(), gz.flatten())).T
    U_sdfvals = sdf(U)
    return U, U_sdfvals

In [ ]:
import gpytoolbox as gpy

# Set up gt
V_gt, F_gt = gpy.read_mesh(mesh_path)
V_gt = gpy.normalize_points(V_gt)

s = 0.9 # 0.75
V_gt *= s/0.5

# Create and abstract SDF function that is the only connection to the shape
sdf = lambda x: gpy.signed_distance(x, V_gt, F_gt)[0]

In [ ]:
n = 30

# Sample SDF values on a regular grid
U, U_sdfvals = sample_positions(n,False,sdf)

In [ ]:
import time

# RFTA
strt = time.time()
R = gpy.reach_for_the_arcs(U, U_sdfvals,parallel=True)
stp  = time.time()
print(f"RFTA (parallel): {stp-strt}s")

In [ ]:
from lie_cone import LieConeSDFReconstruction

strt = time.time()
cone = LieConeSDFReconstruction(np.concatenate([U,U_sdfvals[:,None]],axis=1),
                                    filter_type=3,cut_bbx_factor=1.,filter_results=False)

stp  = time.time()
print(f"Empty spheres rec: {stp-strt}s")

R_lc = cone.V, cone.F

In [ ]:
import polyscope as ps

ps.init()
ps.set_ground_plane_mode("none")

ps.register_surface_mesh("GT", V_gt, F_gt)
ps.register_surface_mesh("RFTA", *R)
ps.register_surface_mesh("LC", *R_lc)

ps.show()